In [2]:
import pandas as pd
import re
import json
from rich import print

from utils import evaluate_ner, basic_post_processing


def parse_ner_output(generated_text, split_by=""):

    segments = generated_text.split("### Entities:" + split_by)
    if len(segments) < 2:
        return []

    after_entities = segments[1].strip()

    entities = []
    for line in after_entities.split("\n"):
        line = line.strip()
        match = re.match(r"^([A-Z]+):\s*(.*)", line)
        if match:
            label, entity_text = match.groups()
            if label in POSSIBLE_ENTITIES:
                entities.append({"label": label, "text": entity_text})

    return json.dumps(entities, ensure_ascii=False)


data_df = pd.read_csv("../data/test.csv")

BASE_PATH = "../results/sft/"
POSSIBLE_ENTITIES = [
    "PERS", "DOC", "QUANT", "ART", "TIME", "JOB",
    "MISC", "PCT", "ORG", "LOC", "PERIOD", "MON", "DATE"
]

In [2]:
files_to_eval = [
    "gemma-2-27b-it_16_ex.csv",
    "gemma-2-27b-it_32_ex.csv",
    "gemma-2-27b-it_64_ex.csv",
    "gemma-3-27b-it_16_ex.csv",
    "gemma-3-27b-it_32_ex.csv",
    "gemma-3-27b-it_64_ex.csv",
    "Qwen2.5-14B-Instruct_16_ex.csv",
    "Qwen2.5-14B-Instruct_32_ex.csv",
    "Qwen2.5-14B-Instruct_64_ex.csv",
]

phi_to_eval = [
    "Phi-4_16_ex.csv",
    "Phi-4_32_ex.csv",
    "Phi-4_64_ex.csv",
]

res = {}
for f_name in files_to_eval:
    df = pd.read_csv(BASE_PATH + f_name)
    data_df["pred"] = df["generated_answer"].apply(parse_ner_output)
    data_df["pred"] = data_df.apply(basic_post_processing, axis=1)
    res[f_name] = evaluate_ner(data_df)["ents_f"]
    
for f_name in phi_to_eval:
    df = pd.read_csv(BASE_PATH + f_name)
    data_df["pred"] = df["generated_answer"].map(lambda x: parse_ner_output(x, "assistant"))
    data_df["pred"] = data_df.apply(basic_post_processing, axis=1)
    res[f_name] = evaluate_ner(data_df)["ents_f"]
     
    
print(json.dumps(res, indent=2))

{
  "gemma-2-27b-it_16_ex.csv": 0.8653431481894565,
  "gemma-2-27b-it_32_ex.csv": 0.8603829160530191,
  "gemma-2-27b-it_64_ex.csv": 0.86366669137816,
  "gemma-3-27b-it_16_ex.csv": 0.8667719852553976,
  "gemma-3-27b-it_32_ex.csv": 0.8787969474786773,
  "gemma-3-27b-it_64_ex.csv": 0.8822429906542056,
  "Qwen2.5-14B-Instruct_16_ex.csv": 0.8512079476179725,
  "Qwen2.5-14B-Instruct_32_ex.csv": 0.8512751071992779,
  "Qwen2.5-14B-Instruct_64_ex.csv": 0.853515625,
  "Phi-4_16_ex.csv": 0.8694021453754408,
  "Phi-4_32_ex.csv": 0.8713910761154855,
  "Phi-4_64_ex.csv": 0.8742874287428744
}

In [6]:
best_files_to_eval = [
    "Qwen2.5-14B-Instruct_Best.csv",
    "gemma-2-27b-it_Best.csv",
    "gemma-3-27b-it_Best.csv",
]

best_res = {}
for f_name in best_files_to_eval:
    df = pd.read_csv(BASE_PATH + f_name)
    data_df["pred"] = df["generated_answer"].apply(parse_ner_output)
    data_df["pred"] = data_df.apply(basic_post_processing, axis=1)
    print(f_name)
    res = evaluate_ner(data_df)
    print(res["ents_f"])
    print([{k: round(v["f"], 3)} for k, v in res["ents_per_type"].items()])


for f_name in ["Phi-4_Best.csv"]:
    df = pd.read_csv(BASE_PATH + f_name)
    data_df["pred"] = df["generated_answer"].map(lambda x: parse_ner_output(x, "assistant"))
    data_df["pred"] = data_df.apply(basic_post_processing, axis=1)
    print(f_name)
    res = evaluate_ner(data_df)
    print(res["ents_f"])
    print([{k: round(v["f"], 3)} for k, v in res["ents_per_type"].items()])

Qwen2.5-14B-Instruct_Best.csv

0.8665516465381441

[
    {'LOC': 0.89},
    {'PCT': 0.977},
    {'PERS': 0.955},
    {'JOB': 0.624},
    {'PERIOD': 0.667},
    {'ORG': 0.898},
    {'MISC': 0.39},
    {'ART': 0.725},
    {'QUANT': 0.812},
    {'DATE': 0.866},
    {'TIME': 0.4},
    {'DOC': 0.479},
    {'MON': 0.95}
]

gemma-2-27b-it_Best.csv

0.8861648852679572

...

0.8879245847673201

[
    {'LOC': 0.929},
    {'PCT': 0.989},
    {'PERS': 0.965},
    {'JOB': 0.642},
    {'PERIOD': 0.747},
    {'ORG': 0.923},
    {'MISC': 0.35},
    {'QUANT': 0.897},
    {'DATE': 0.906},
    {'ART': 0.716},
    {'TIME': 0.632},
    {'DOC': 0.514},
    {'MON': 0.953}
]

Phi-4_Best.csv

0.8817332835263354

[
    {'LOC': 0.907},
    {'PCT': 0.966},
    {'PERS': 0.967},
    {'JOB': 0.638},
    {'PERIOD': 0.714},
    {'ORG': 0.911},
    {'DOC': 0.464},
    {'MISC': 0.314},
    {'ART': 0.774},
    {'QUANT': 0.833},
    {'DATE': 0.885},
    {'TIME': 0.571},
    {'MON': 0.958}
]